## 1. Import modules

- %load_ext sql imports "sql magic" that allows us to use write sql code.

- Pandas allows us to leverage python code to simplify the "extract" step of our ETL.

- sqlite3 allows us to establish a connection with our database in order to leverage the use of python code.


In [1]:
%load_ext sql
import pandas as pd
import sqlite3 as sql

## 2. Create database

- Use sql to create 'CourseData.db'.

- Create a python variable called 'conn' to interact with our database using python code. 


In [2]:
%sql sqlite:///CourseData.db
conn = sql.connect('CourseData.db')

![Team %%sql Project ERD](CourseDataERD.png)

## 3. Create tables matching ERD infrastructure

- Create to match the CourseDataERD schema.

- Ensure there are no truncation issues (Dropping tables before creating).


In [3]:
%%sql

-- Create PROGRAMS table
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

-- Create CATALOG_COURSES table
DROP TABLE IF EXISTS CATALOG_COURSES;
CREATE TABLE CATALOG_COURSES (
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID INTEGER,
    CourseTitle TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
);
CREATE INDEX index_catalog_courses_alt_key on CATALOG_COURSES(CatalogYear,CatalogID);

-- Create COURSE_INSTRUCTORS table
DROP TABLE IF EXISTS COURSE_INSTRUCTORS;
CREATE TABLE COURSE_INSTRUCTORS (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);
CREATE INDEX index_course_instructors_alt_key on COURSE_INSTRUCTORS(Name);

-- Course COURSE_OFFERINGS table
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS (
    CourseOfferingID INTEGER PRIMARY KEY,
    CourseID INTEGER,
    CatalogID TEXT NOT NULL,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Credits REAL,
    Title TEXT NOT NULL,
    Timecodes TEXT,
    PrimaryInstructorID INTEGER,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL
);
CREATE INDEX index_course_offerings_alt_key on COURSE_OFFERINGS(Term,CatalogID,Section);

-- Create COURSE_LOCATIONS table
DROP TABLE IF EXISTS COURSE_LOCATIONS;
CREATE TABLE COURSE_LOCATIONS (
    LocationID INTEGER PRIMARY KEY,
    LocationCode TEXT NOT NULL
);


-- Create COURSE_MEETINGS table
DROP TABLE IF EXISTS COURSE_MEETINGS;
CREATE TABLE COURSE_MEETINGS (
    CourseMeetingID INTEGER PRIMARY KEY,
    CourseOfferingID INTEGER NOT NULL,
    LocationID INTEGER NOT NULL,
    StartDateTime TEXT NOT NULL,
    EndDateTime TEXT NOT NULL
);
-- Conversion table for con
-- Create TERM_TO_CATALOG_YEAR table
DROP TABLE IF EXISTS TERM_TO_CATALOG_YEAR;
CREATE TABLE TERM_TO_CATALOG_YEAR (
    CatalogYear TEXT NOT NULL,
    Term TEXT NOT NULL
);

INSERT INTO TERM_TO_CATALOG_YEAR (Term, CatalogYear) VALUES 
('Fall2017','2017_2018'),('Winter2018','2017_2018'),('Spring2018','2017_2018'),('Summer2018','2017_2018'),('Fall2018','2018_2019'),
('Winter2019','2018_2019'),('Spring2019','2018_2019');

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
7 rows affected.


[]

## 4. Delete & Drop 'IMPORT' tables

- This enables us to re-create our database from scratch if needed, deleting and removing any populated 'IMPORT' tables from prior runs.


In [4]:
%%sql
DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;
DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;
DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseData.db
(sqlite3.OperationalError) no such table: IMPORT_CATALOG_COURSES
[SQL: DELETE FROM IMPORT_CATALOG_COURSES;]
(Background on this error at: http://sqlalche.me/e/e3q8)


## 5. Use loops to populate 'IMPORT' tables

- __IMPORT_CATALOG_COURSES__ uses a list with the available catalog years to loop through the available course catalog csv files and upload them to our sql table.

- __IMPORT_COURSE_OFFERINGS__ uses a list of semesters (also known as 'Term') to loop through the available course offerings csv files and upload them to our sql table.

- __IMPORT_COURSE_MEETINGS__ uses the same list of semesters to loop through the course meetings csv files and upload them to our sql table.


In [5]:
catalog_years = ['2017_2018', '2018_2019']

for catalog_year in catalog_years:
    file_path = 'SourceData/Catalogs/CourseCatalog' + catalog_year + '.csv'
    df_catalogs = pd.read_csv(file_path)
    df_catalogs['cat_year'] = catalog_year # Add column with 'catalog_year' into table
    df_catalogs.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)
    
semesters = ['Fall2014','Winter2015','Spring2015','Summer2015','Fall2015','Winter2016',
             'Spring2016','Summer2016','Fall2016','Winter2017','Spring2017','SpringBreak2017',
             'Summer2017','Fall2017','Winter2018','Spring2018','Summer2018','Fall2018',
             'Spring2019']

for semester in semesters:
    file_path = 'SourceData/' + semester + '/courses.csv'
    df_course_offerings = pd.read_csv(file_path)
    df_course_offerings.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False)
    
    file_path = 'SourceData/' + semester + '/course_meetings.csv'
    df_course_meetings = pd.read_csv(file_path)
    df_course_meetings.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)
    

## 6. Check for duplicate data
- Use sql queries to compare Raw vs. Distinct counts of data within each 'IMPORT' table.

In [6]:
%%sql
-- Check for duplicate values
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'Raw Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'Distinct Count';

 * sqlite:///CourseData.db
Done.


Raw Count,Distinct Count
4440,4440


In [7]:
%%sql
-- Check for duplicate values
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'Raw Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'Distinct Count';

 * sqlite:///CourseData.db
Done.


Raw Count,Distinct Count
15937,15937


In [8]:
%%sql
-- Check for duplicate values
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'Raw Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'Distinct Count';

 * sqlite:///CourseData.db
Done.


Raw Count,Distinct Count
284907,284847


#### Duplicates identified within IMPORT_COURSE_MEETINGS table.

- The below query calculates the total number of duplicates as 60.

- With this knowledge, we now know that we must load the data as 'Distinct' into our database when retrieving fields from the IMPORT_COURSE_MEETINGS table.


In [9]:
%%sql
SELECT COUNT(*) - (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) AS 'TotalDuplicateValues'
FROM IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseData.db
Done.


TotalDuplicateValues
60


## 7. Populate ERD tables

- Populate the tables that were created mirroring our ERD using the 'IMPORT' tables.

- Ensure there are no truncation issues by deleting the data from the tables prior to populating them.

In [10]:
%%sql
DELETE FROM COURSE_INSTRUCTORS;

INSERT INTO COURSE_INSTRUCTORS (Name)
SELECT DISTINCT primary_instructor
FROM IMPORT_COURSE_OFFERINGS
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [11]:
%%sql
DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (ProgramCode,ProgramName)
SELECT DISTINCT program_code,program_name 
FROM IMPORT_CATALOG_COURSES
ORDER BY program_code;

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [12]:
%%sql
DELETE FROM CATALOG_COURSES;

INSERT INTO CATALOG_COURSES (CatalogYear,ProgramID,CatalogID,CourseTitle,Credits,Prereqs,Coreqs,Fees,Attributes,Description)
SELECT DISTINCT cat_year, ProgramID,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
FROM IMPORT_CATALOG_COURSES 
    JOIN PROGRAMS ON (program_code = ProgramCode);

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [13]:
%%sql
DELETE FROM COURSE_OFFERINGS;

INSERT INTO COURSE_OFFERINGS (CourseID,Term,CRN,CatalogID,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity, Actual, Remaining)
SELECT DISTINCT CourseID,term,crn,catalog_id,section,IMPORT_COURSE_OFFERINGS.credits,title,timecodes,InstructorID,cap,act,rem 
FROM IMPORT_COURSE_OFFERINGS 
    LEFT JOIN COURSE_INSTRUCTORS ON (primary_instructor=COURSE_INSTRUCTORS.Name)
    LEFT JOIN TERM_TO_CATALOG_YEAR USING (Term)
    LEFT JOIN CATALOG_COURSES ON (catalog_id = CatalogID AND CATALOG_COURSES.CatalogYear = TERM_TO_CATALOG_YEAR.CatalogYear)
;

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [14]:
%%sql 
DELETE FROM COURSE_LOCATIONS;

INSERT INTO COURSE_LOCATIONS (LocationCode)
SELECT DISTINCT Location 
FROM IMPORT_COURSE_MEETINGS
ORDER BY Location

 * sqlite:///CourseData.db
0 rows affected.
207 rows affected.


[]

In [15]:
%%sql
DELETE FROM COURSE_MEETINGS;

INSERT INTO COURSE_MEETINGS (CourseOfferingID,LocationID,StartDateTime,EndDateTime)
SELECT DISTINCT COURSE_OFFERINGS.CourseOfferingID, LocationID,start,end
FROM IMPORT_COURSE_MEETINGS 
    JOIN COURSE_OFFERINGS USING (Term,CRN)
    LEFT JOIN COURSE_LOCATIONS ON (IMPORT_COURSE_MEETINGS.Location = COURSE_LOCATIONS.LocationCode);

 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.


[]

## 9. Clear storage space

- Delete 'IMPORT' tables to recover storage space.

In [16]:
%%sql

DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;

DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;
DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;

vacuum;

 * sqlite:///CourseData.db
4440 rows affected.
15937 rows affected.
284907 rows affected.
Done.
Done.
Done.
Done.


[]